In [1]:
import sys
import os
home=os.getcwd()
sys.path.append(home+'/Functions')
from ChargeDataSet import *
from ShowDF import *
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
DataSetName='12111.mzML'

In [3]:
DataSet=ChargeDataSet(DataSetName)

In [ ]:
I can predict the ID with the RT, I just need to make a quick correlation at the beggining 

In [22]:
IDvec=[]
RTvec=[]
c=0
for Spec in DataSet:
    #print(x)
    #Spec=DataSet[int(x)]
    if Spec.getMSLevel()==1:
        IDvec.append(c)        
        RTvec.append(Spec.getRT())
        c+=1
#plt.plot(RTvec,IDvec) 
#print(IDvec)

In [5]:
FirstID=1099
LastID=1139
#My next step is to melt a certain number of spectrum to make the clustering easier. A small error on the normalization will be included, as the intensity on each spectrum is a function of the concentration as well

In [68]:
IDvec=MS1IDs(DataSet)

In [69]:
Spectrum=JoinSpectra(IDvec=IDvec,DataSet=DataSet,FirstID=FirstID,LastID=LastID)

In [74]:
CentroidsValley(spectra=Spectrum.copy(),Centroids=c.copy())

In [75]:
Centroids=SpectraCentroids(spectra=Spectrum.copy(),SafetyFactor=4)
allFeatures=ALLFeatures(Centroids,allFeatures=[])

In [81]:
from scipy.stats import t
import numpy as np
def FeaturesStats(Signals,alpha=0.01): #Isotopomer refers to either the most abundant isotopomer, or the one with 18O
    SumIntens=sum(Signals[:,1])
    RelativeInt=Signals[:,1]/SumIntens
    AverageMZ=sum(Signals[:,0]*RelativeInt)
    l=len(Signals[:,1])    
    VarianMZ=sum(RelativeInt*(Signals[:,0]-AverageMZ)**2)*l/(l-1)  
    StdMZ=np.sqrt(VarianMZ)
    featuresStats=[AverageMZ,StdMZ]    
    return featuresStats

In [76]:
FeaturesData(Centroids=Centroids,spectra=Spectrum,allFeatures=allFeatures)

In [ ]:
def UpdateBorders(Centroids,allFeatures):
    